# Customer Segmentation Using K-Means Clustering

In [1]:
import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib as plt
import plotly.express as px
import sklearn

### Motivations: 
Customer segmentation is the fundamental basis of recommendations to improve customer satisfaction and retention. Clustering customers help to identify users with similar behaviors, patterns or beliefs which helps companies provide accurate personalized services for the differnt kinds of customers. Clustering not only aids in providing a seamless experience for the users, but it also helps companies identify opportunities for growth and investment, as well as to distrubute resources according to each identified group.

For this project we are going to use K-Means clustering, with marketing data from [kaggle](https://www.kaggle.com/datasets/rodsaldanha/arketing-campaign) to create customer segmentation clusters. 

### Types of Clustering:
Clustering can be done in different ways, depending on business intentions and motivations. 

* **Geographic segmentation:** Clustering customers based on location. Can be used for projects such as identifying traffic patterns or identifying highest grossing products based on location.

* **Demographic segmentation:** Clustering individuals by personal similarities and motivations. For example, you clustering individuals who regularly purchase diapers as parents vs those who do not as non-parents. Other examples include identifying the age and gender of the most loyal costumers, which can then help focus initiatives on retaining those customers. 

* **Psychographic segmentation:** Clustering individuals based on interests, beliefs and values. This can mean grouping individuals with similar taste in music, running patters or clothing brands.

* **Behavioral segmentation:** Clustering individuals based on their habits or actions such as app-usage or store-visit patterns. Can be used to identify involvement and sense of loyalty.

* **Needs-based segmentation:** Clustering individuals based on needs. For example, grouping customers that require home delivery options as a must-have.

* **Value-based segmentation:** Clustering customers based on their spending patterns, revenue and loyalty spending for the company. 

* **Technographic segmentation:** Clustering individuals by the way they use and access technology to reach services, or buy products. For example, identifying those that use desktop, mobile or external applications to complete transactions or access services. 



### Data:
Below we explore the different columns in the data and also group them into the different cluster-types we can formulate from the columns. 

#### Demographic segmentation: 
ID: Unique identifyer for each customer in the data <br>
Year_Birth: Year the customer was born <br>
Education: Customer's highest level of education <br>
Maritual Status: Whether customer is married or not  <br>
Income: Yearly income  <br>
Kidhome: Are there kids below 13 in the customer's home <br>
Teenhome: Are there kids
Dt_customer: Date the customer signed up to purchase from the company <br>
Recency: How long it has been (days) since the customer visited the store <br> above 13 in the home <br>


#### Needs based segmentation / Value based segmentation:
The following columns contain information regarding spenditure in the last two years
MntWines: Amount spent on wine   <br>
MntFruits: Amount spent on food  <br>
MntMeatProducts: Amount spent on meat  <br>
MntFishProducts: Amount spent on fish <br>
MntSweetProducts: Amount spent on sweets <br>
MntGoldProds: Amount spent on gold products <br>

#### Technographic segmentation:
NumWebPurchases: Number of purchases made via company's website  <br>
NumCatalogPurchases: Number purchases made via company's catalog <br>
NumStorePurchases: Number of purchases made in store <br>
NumWebVisitsMonth: Number of times the customer visited the store in the last month<br>

#### Behavioral segmentation:
The following columns identify the customer's acceptance to advertisement campaigns, 1 if yes, 0 if not
AcceptedCmp1: Customers who accepted campaign's first attempt  <br>
AcceptedCmp2: Customers who accepted campaign's second attempt <br>
AcceptedCmp3: Customers who accepted campaign's third attempt<br>
AcceptedCmp4: Customers who accepted campaign's fourth attempt<br>
AcceptedCmp5: Customers who accepted campaign's fifth attempt <br>
Complain: Captures whether the customer complained(1) or not(0) in the past two years<br>


#### Value based segmentation:
Z_CostContact  <br>
Z_Revenue <br>
Response <br>
NumDealsPurchases: Purchases made by the customer using promotion/sale  <br>


In [2]:
df = pd.read_csv('marketing_campaign.csv', sep='\t')
df.head(50)

,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,...,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response
0,5524,1957,Graduation,Single,58138.0,0,0,04-09-2012,58,635,...,7,0,0,0,0,0,0,3,11,1
1,2174,1954,Graduation,Single,46344.0,1,1,08-03-2014,38,11,...,5,0,0,0,0,0,0,3,11,0
2,4141,1965,Graduation,Together,71613.0,0,0,21-08-2013,26,426,...,4,0,0,0,0,0,0,3,11,0
3,6182,1984,Graduation,Together,26646.0,1,0,10-02-2014,26,11,...,6,0,0,0,0,0,0,3,11,0
4,5324,1981,PhD,Married,58293.0,1,0,19-01-2014,94,173,...,5,0,0,0,0,0,0,3,11,0
5,7446,1967,Master,Together,62513.0,0,1,09-09-2013,16,520,...,6,0,0,0,0,0,0,3,11,0
6,965,1971,Graduation,Divorced,55635.0,0,1,13-11-2012,34,235,...,6,0,0,0,0,0,0,3,11,0
7,6177,1985,PhD,Married,33454.0,1,0,08-05-2013,32,76,...,8,0,0,0,0,0,0,3,11,0
8,4855,1974,PhD,Together,30351.0,1,0,06-06-2013,19,14,...,9,0,0,0,0,0,0,3,11,1
9,5899,1950,PhD,Together,5648.0,1,1,13-03-2014,68,28,...,20,1,0,0,0,0,0,3,11,0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2240 entries, 0 to 2239
Data columns (total 29 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ID                   2240 non-null   int64  
 1   Year_Birth           2240 non-null   int64  
 2   Education            2240 non-null   object 
 3   Marital_Status       2240 non-null   object 
 4   Income               2216 non-null   float64
 5   Kidhome              2240 non-null   int64  
 6   Teenhome             2240 non-null   int64  
 7   Dt_Customer          2240 non-null   object 
 8   Recency              2240 non-null   int64  
 9   MntWines             2240 non-null   int64  
 10  MntFruits            2240 non-null   int64  
 11  MntMeatProducts      2240 non-null   int64  
 12  MntFishProducts      2240 non-null   int64  
 13  MntSweetProducts     2240 non-null   int64  
 14  MntGoldProds         2240 non-null   int64  
 15  NumDealsPurchases    2240 non-null   i

## Data Cleaning:


Before we begin cleaning our columns, let's investigate some columns that we are not sure about: 
* Z_CostContract
* Z_Revenue
* Reponse

In [4]:
print(df['Z_CostContact'].describe())
print(df['Z_CostContact'].unique())
print()

print(df['Z_Revenue'].describe())
print(df['Z_Revenue'].unique())
print()

print(df['Response'].describe())
print(df['Response'].unique())

count    2240.0
mean        3.0
std         0.0
min         3.0
25%         3.0
50%         3.0
75%         3.0
max         3.0
Name: Z_CostContact, dtype: float64
[3]

count    2240.0
mean       11.0
std         0.0
min        11.0
25%        11.0
50%        11.0
75%        11.0
max        11.0
Name: Z_Revenue, dtype: float64
[11]

count    2240.000000
mean        0.149107
std         0.356274
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max         1.000000
Name: Response, dtype: float64
[1 0]


These values don't seem to be very useful for our project so we will begin by dropping them from our dataset.

In [10]:
df = df.drop(['Z_CostContact', 'Z_Revenue', 'Response'], axis=1)

Let's determine the rows that are missing all values 

In [37]:
null_vals = df.loc[df.isnull().any(axis=1)]
null_vals

,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,...,NumWebPurchases,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain


Identifying distribution of numerical values:

In [63]:
num_features = df.select_dtypes('int64')
# print(num_features)
for feature in num_features:
    fig = px.histogram(df[feature],
    width=600, height=400
    )
    fig.show()


## Feature engineering:
* **Age:** Using Year_Birth feature to create an age column for each unique customer <br>
* **Loyalty_since:** We'll be using the Dt_customer to determine the time the customer has been a member of the company based from the most recent input in the dataset <br>
* **Total_goods:** We'll be adding all the products to create a general amount of goods spent by the customer <br>
* **Edu_leve_US:** Updating this column to reflect standard US expressions. In the dataset, "Graduation" means that the customer completed a Bachelor's degree, and "2nd Cycle" means Master's degree <br>
